In [1]:
import torch
import numpy as np
from torchvision import datasets
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F

In [2]:
transform = transforms.ToTensor()
train_data = datasets.MNIST(root='data', train=True,
                                   download=True, transform=transform)
test_data = datasets.MNIST(root='data', train=False,
                                  download=True, transform=transform)

In [3]:
imgs = torch.stack([img_t for img_t, _ in train_data], dim=3)

In [4]:
imgs.shape

torch.Size([1, 28, 28, 60000])

In [5]:
imgs.view(1, -1).mean(dim=1)

tensor([0.1307])

In [6]:
imgs.view(1, -1).std(dim=1)

tensor([0.3081])

In [3]:
train_data = datasets.MNIST(root='data', train=True,
                                   download=True, transform=transforms.Compose([
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,),(0.3081,))
                                   ]))
test_data = datasets.MNIST(root='data', train=False,
                                   download=True, transform=transforms.Compose([
                                       transforms.ToTensor(),
                                       transforms.Normalize((0.1307,),(0.3081,))
                                    ]))

In [4]:
train_loader = torch.utils.data.DataLoader(train_data, batch_size=100)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=100)

In [5]:
iter(train_loader).next()[0].size()

torch.Size([100, 1, 28, 28])

In [26]:
# definir la arquitectura
class ConvolutionalNN(nn.Module):
    def __init__(self):
        super(ConvolutionalNN, self).__init__()
        ## convolutional layers ##
        self.t_conv_1 = nn.Conv2d(1, 32, kernel_size = 3, padding = 1)
        self.t_conv_2 = nn.Conv2d(32, 64, kernel_size = 3, padding = 1)
        self.max_pool_1 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        self.max_pool_2 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        self.linear_1 = nn.Linear(64 * 7 * 7,2500)
        self.linear_2 = nn.Linear(2500,10)

    def forward(self, x):
        ## NeuralNet ##
        result = self.t_conv_1(x)
        result = F.relu(result)
        result = self.max_pool_1(result)
        result = self.t_conv_2(result)
        result = F.relu(result)
        result = self.max_pool_2(result)
        result = result.reshape(100, -1)
        result = self.linear_1(result)
        result = self.linear_2(result)
        return result
model = ConvolutionalNN()

In [28]:
## TRAINING LOOP ##IMPORTANTE: SE PUEDE CARGAR EL ARCHIVO CON LA LINEA INFERIOR PARA EVITAR REENTRENAR EL MODELO TARDA 
##1 HORA Y 30 MINUTOS DE ENTRENAMIENTO
n_epochs = 20
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
total_step = len(train_loader)
loss_list = []
acc_list = []
for epoch in range(n_epochs):
    for i, (images, labels) in enumerate(train_loader):
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss_list.append(loss.item())
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        total = labels.size(0)
        _, predicted = torch.max(outputs.data, 1)
        correct = (predicted == labels).sum().item()
        acc_list.append(correct / total)
        if (i + 1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Accuracy: {:.2f}%'
                  .format(epoch + 1, n_epochs, i + 1, total_step, loss.item(),
                          (correct / total) * 100))

Epoch [1/20], Step [100/600], Loss: 0.0365, Accuracy: 99.00%
Epoch [1/20], Step [200/600], Loss: 0.0579, Accuracy: 98.00%
Epoch [1/20], Step [300/600], Loss: 0.0854, Accuracy: 97.00%
Epoch [1/20], Step [400/600], Loss: 0.0533, Accuracy: 99.00%
Epoch [1/20], Step [500/600], Loss: 0.1461, Accuracy: 94.00%
Epoch [1/20], Step [600/600], Loss: 0.2548, Accuracy: 95.00%
Epoch [2/20], Step [100/600], Loss: 0.0103, Accuracy: 100.00%
Epoch [2/20], Step [200/600], Loss: 0.0668, Accuracy: 98.00%
Epoch [2/20], Step [300/600], Loss: 0.0661, Accuracy: 98.00%
Epoch [2/20], Step [400/600], Loss: 0.0261, Accuracy: 99.00%
Epoch [2/20], Step [500/600], Loss: 0.1068, Accuracy: 95.00%
Epoch [2/20], Step [600/600], Loss: 0.2290, Accuracy: 99.00%
Epoch [3/20], Step [100/600], Loss: 0.0073, Accuracy: 100.00%
Epoch [3/20], Step [200/600], Loss: 0.0080, Accuracy: 99.00%
Epoch [3/20], Step [300/600], Loss: 0.0146, Accuracy: 99.00%
Epoch [3/20], Step [400/600], Loss: 0.0439, Accuracy: 98.00%
Epoch [3/20], Step [50

In [30]:
model.eval() 
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()
    print('Test Accuracy: {} %'.format((correct / total) * 100))

Test Accuracy: 98.79 %


In [29]:
#GUARDAR EL ESTADO
#torch.save(model.state_dict(),'conv_net_model.ckpt')

In [ ]:
#CARGAR EL MODELO
##Utilizar esta linea para cargar el modelo
model = ConvolutionalNN()
model.load_state_dict(torch.load('conv_net_model.ckpt'))
model.eval()